In [32]:
from IPython import get_ipython
from IPython.display import display
import datetime
import sqlite3
import tkinter as tk 
from tkinter import scrolledtext
from tkinter import messagebox


In [13]:
# Database functions
def init_db():
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS expenses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            date TEXT NOT NULL,
            amount REAL NOT NULL,
            category TEXT NOT NULL,
            description TEXT,
            FOREIGN KEY (user_id) REFERENCES users(id)
        )
    """)
    conn.commit()
    conn.close()

In [14]:
def register_user(username):
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users (username) VALUES (?)", (username,))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()

In [15]:
def get_user_id(username):
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE username=?", (username,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

In [16]:
def add_expense(username, amount, category, description=""):
    user_id = get_user_id(username)
    if not user_id:
        if register_user(username):
            user_id = get_user_id(username)
        else:
            return False
    
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO expenses (user_id, date, amount, category, description)
        VALUES (?, ?, ?, ?, ?)
    """, (user_id, date, amount, category, description))
    conn.commit()
    conn.close()
    return True

In [17]:
def get_expenses(username, category_filter=None):
    user_id = get_user_id(username)
    if not user_id:
        return None
    
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    
    if category_filter:
        cursor.execute("""
            SELECT date, amount, category, description 
            FROM expenses 
            WHERE user_id=? AND category=?
            ORDER BY date DESC
        """, (user_id, category_filter))
    else:
        cursor.execute("""
            SELECT date, amount, category, description 
            FROM expenses 
            WHERE user_id=?
            ORDER BY date DESC
        """, (user_id,))
    
    expenses = cursor.fetchall()
    conn.close()
    return expenses

In [33]:
from tkinter import ttk 
# GUI Application
class ExpenseTrackerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Expense Tracker")
        self.root.geometry("700x600")
        
        # Initialize database
        init_db()
        
        # Create widgets
        self.create_widgets()
    
    def create_widgets(self):
        # Input Frame
        input_frame = tk.LabelFrame(self.root, text="Add New Expense", padx=10, pady=10)
        input_frame.pack(pady=10, fill="x", padx=10)
        
        # Username
        tk.Label(input_frame, text="Username:").grid(row=0, column=0, sticky="e", padx=5, pady=5)
        self.username_entry = tk.Entry(input_frame, width=30)
        self.username_entry.grid(row=0, column=1, padx=5, pady=5)
        
        # Amount
        tk.Label(input_frame, text="Amount:").grid(row=1, column=0, sticky="e", padx=5, pady=5)
        self.amount_entry = tk.Entry(input_frame, width=30)
        self.amount_entry.grid(row=1, column=1, padx=5, pady=5)
        
        # Category
        tk.Label(input_frame, text="Category:").grid(row=2, column=0, sticky="e", padx=5, pady=5)
        self.category_entry = ttk.Combobox(input_frame, width=28)
        self.category_entry['values'] = ("Food", "Transport", "Entertainment", "Bills", "Shopping", "Other")
        self.category_entry.grid(row=2, column=1, padx=5, pady=5)
        
        # Description
        tk.Label(input_frame, text="Description:").grid(row=3, column=0, sticky="e", padx=5, pady=5)
        self.description_entry = tk.Entry(input_frame, width=30)
        self.description_entry.grid(row=3, column=1, padx=5, pady=5)
        
        # Buttons
        button_frame = tk.Frame(input_frame)
        button_frame.grid(row=4, column=0, columnspan=2, pady=10)
        
        tk.Button(button_frame, text="Add Expense", command=self.add_expense).pack(side="left", padx=5)
        tk.Button(button_frame, text="View Expenses", command=self.view_expenses).pack(side="left", padx=5)
        
        # Filter Frame
        filter_frame = tk.LabelFrame(self.root, text="Filter Expenses", padx=10, pady=10)
        filter_frame.pack(pady=10, fill="x", padx=10)
        
        tk.Label(filter_frame, text="Filter by Category:").pack(side="left", padx=5)
        self.category_filter = ttk.Combobox(filter_frame, width=15)
        self.category_filter['values'] = ("All", "Food", "Transport", "Entertainment", "Bills", "Shopping", "Other")
        self.category_filter.current(0)
        self.category_filter.pack(side="left", padx=5)
        tk.Button(filter_frame, text="Apply Filter", command=self.view_expenses).pack(side="left", padx=5)
        
        # Display Frame
        display_frame = tk.LabelFrame(self.root, text="Expense History", padx=10, pady=10)
        display_frame.pack(pady=10, fill="both", expand=True, padx=10)
        
        self.expenses_text = scrolledtext.ScrolledText(
            display_frame, 
            width=80, 
            height=20,
            wrap=tk.WORD
        )
        self.expenses_text.pack(fill="both", expand=True, padx=5, pady=5)
    
    def add_expense(self):
        username = self.username_entry.get()
        amount = self.amount_entry.get()
        category = self.category_entry.get()
        description = self.description_entry.get()
        
        if not all([username, amount, category]):
            messagebox.showerror("Error", "Username, amount and category are required!")
            return
        
        try:
            amount = float(amount)
            if add_expense(username, amount, category, description):
                messagebox.showinfo("Success", "Expense added successfully!")
                # Clear fields
                self.amount_entry.delete(0, tk.END)
                self.description_entry.delete(0, tk.END)
            else:
                messagebox.showerror("Error", "Failed to add expense!")
        except ValueError:
            messagebox.showerror("Error", "Invalid amount!")
    
    def view_expenses(self):
        username = self.username_entry.get()
        if not username:
            messagebox.showerror("Error", "Please enter a username!")
            return
        
        category_filter = self.category_filter.get()
        if category_filter == "All":
            category_filter = None
        
        expenses = get_expenses(username, category_filter)
        
        self.expenses_text.delete(1.0, tk.END)
        
        if expenses is None:
            self.expenses_text.insert(tk.END, f"User '{username}' not found!")
            return
        
        if not expenses:
            self.expenses_text.insert(tk.END, "No expenses found!")
            return
        
        # Display header
        header = f"{'Date':<12}{'Amount':>10}{'Category':>15}{'Description':>20}\n"
        self.expenses_text.insert(tk.END, header)
        self.expenses_text.insert(tk.END, "-"*60 + "\n")
        
        # Display expenses
        total = 0
        for expense in expenses:
            date, amount, category, description = expense
            total += amount
            row = f"{date:<12}{amount:>10.2f}{category:>15}{description:>20}\n"
            self.expenses_text.insert(tk.END, row)
        
        # Display total
        self.expenses_text.insert(tk.END, "-"*60 + "\n")
        self.expenses_text.insert(tk.END, f"{'Total:':<12}{total:>10.2f}\n")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ExpenseTrackerApp(root)
    root.mainloop()